# Drift - Comparing to Bmad

Simple 1 m drift.

In [ ]:
import numpy as np
import impact.z as IZ
from impact.z.interfaces.bmad import export_particles as tao_export_particles
from pytao import Tao

from pmd_beamphysics import single_particle
from pmd_beamphysics.units import mec2
import matplotlib.pyplot as plt

from impact.tests.z.conftest import bmad_files

In [ ]:
x0 = 0.001
y0 = 0.002
z0 = 0  # By definition
t0 = 0.003
px0 = 1e6
py0 = 2e6
energy0 = 10e6
pz0 = np.sqrt(energy0**2 - px0**2 - py0**2 - mec2**2)

P0 = single_particle(px=px0, py=py0, pz=pz0, x=x0, y=y0, z=z0, t=t0)

In [ ]:
!cat $bmad_files/drift.bmad

In [ ]:
tao = Tao(lattice_file=bmad_files / "drift.bmad", plot="mpl")

In [ ]:
tao.plot("beta", include_layout=False)

In [ ]:
P0.write("drift_initial_particles.h5")
tao.cmds(
    [
        "set beam_init position_file = drift_initial_particles.h5",
        f"set beam_init n_particle = {len(P0)}",
        f"set beam_init bunch_charge = {P0.charge}",
        "set beam_init saved_at = beginning d",
        "set global track_type = single",
        "set global track_type = beam",
    ]
)

In [ ]:
tao.plot("beta", include_layout=False)

In [ ]:
# TODO write header showing repr as well
input = IZ.ImpactZInput.from_tao(tao)

In [ ]:
# Verify the ParticleGroup we dumped from Tao is the same one we specify as initial_particles
assert input.initial_particles == P0

In [ ]:
input.space_charge_off()
# input.space_charge_on(current=1.0)

In [ ]:
I = IZ.ImpactZ(input, use_temp_dir=False, workdir="./tmp", initial_particles=P0)

In [ ]:
I.input.reference_frequency = 1
I.input.n_particle = len(P0)

In [ ]:
output = I.run(verbose=True)

In [ ]:
I.input

In [ ]:
I.output

In [ ]:
Pin = I.output.particles["initial_particles"]

In [ ]:
# Bmad
P0.data

In [ ]:
import pathlib
from impact.z import ImpactZParticles

P0_z_written = ImpactZParticles.from_file(pathlib.Path(I.workdir) / "particle.in")
P0_written = P0_z_written.to_particle_group(
    reference_frequency=I.input.reference_frequency,
    reference_kinetic_energy=I.input.reference_kinetic_energy,
    phase_reference=0,
)
P0_written.data

In [ ]:
# This is what ImpactZ writes as its initial particles
Pin.data

In [ ]:
# Verify that ImpactZ read and wrote our initial particles correctly:
assert P0 == Pin

In [ ]:
P_raw = I.output.particles_raw["final_particles"]
P = I.output.particles["final_particles"]

In [ ]:
P_raw

In [ ]:
P.data

In [ ]:
Ptao = tao_export_particles(tao, "END")

In [ ]:
Ptao.data

In [ ]:
stats = I.output.stats
z = stats.z
x = stats.mean_x
y = stats.mean_y

x_tao = tao.bunch_comb("x")
y_tao = tao.bunch_comb("y")
s_tao = tao.bunch_comb("s")

fig, axes = plt.subplots(2, figsize=(8, 4))
ax = axes[0]
ax.plot(z, x, label="Impact-Z")
ax.plot(s_tao, x_tao, "--", label="Tao")
ax.set_ylabel(r"$x$ (m)")


ax = axes[1]
ax.plot(z, y, label="Impact-Z")
ax.plot(s_tao, y_tao, "--", label="Tao")
ax.set_ylabel(r"$y$ (m)")

ax.set_xlabel(r"$s$ (m)")

plt.legend();